<a href="https://colab.research.google.com/github/dcshapiro/seriously-a-repo-just-to-upload-one-file-for-an-article/blob/main/digitalocean_serverless_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Outline
Author: Daniel Shapiro, PhD. CTO @ [http://lemay.ai](Lemay.ai) 
1. Install doctl
2. Authenticate to digitalocean using doctl
3. Install serverless in doctl, so we can make functions
4. Connect to your digitalocean sandbox
5. Create a digitalocean python function from a template
6. Create a requirements file
7. Fit a regression model on some data
8. Create a build script
9. Create inference pipeline
10. Build and test your digitalocean function

# Install doctl


In [1]:
!curl -sL https://github.com/digitalocean/doctl/releases/download/v1.76.0/doctl-1.76.0-linux-amd64.tar.gz | tar -xzv

doctl


In [2]:
!sudo mv doctl /usr/local/bin

# Authenticate to digitalocean using doctl

In [3]:
!doctl auth init –-context lemay_ai 

Please authenticate doctl for use with your DigitalOcean account. You can generate a token in the control panel at https://cloud.digitalocean.com/account/api/tokens

Enter your access token: 
Validating token... OK



# Install serverless in doctl, so we can make functions

In [4]:
!doctl serverless install

Downloading...Unpacking...Installing...Cleaning up...
Done


Create a directory to work in

In [5]:
!mkdir -p functions

In [6]:
%cd functions

/content/functions


In [7]:
!pwd

/content/functions


# Connect to your digitalocean sandbox

In [8]:
!doctl sandbox connect

Connected to function namespace 'fn-36f74ea7-e6c1-44dc-b004-b142654d8b0b' on API host 'https://faas-tor1-70ca848e.doserverless.co'



# Create a digitalocean python function from a template

In [9]:
!doctl sandbox init . -l python

A local sandbox area '.' was created for you.
You may deploy it by running the command shown on the next line:
  doctl sandbox deploy .



Deploy your new blank project

In [10]:
!doctl sandbox deploy . --remote-build

Deploying '/content/functions'
  to namespace 'fn-36f74ea7-e6c1-44dc-b004-b142654d8b0b'
  on host 'https://faas-tor1-70ca848e.doserverless.co'
Deployment status recorded in '.nimbella'

Deployed functions ('doctl sbx fn get <funcName> --url' for URL):
  - sample/hello


Generate the URL for this project so that you can see it in the browser. This is also the URL you can post parameters to, as we will see later on.


In [11]:
!doctl sbx fn get sample/hello --url

https://faas-tor1-70ca848e.doserverless.co/api/v1/web/fn-36f74ea7-e6c1-44dc-b004-b142654d8b0b/sample/hello


Invoke this blank project from the command line, so that we don't need to tab over to the browser.

In [12]:
!doctl sls fn invoke sample/hello

{
  "body": "Hello stranger!"
}


# Create a requirements file
That was boring. Let's make something real.

If you have python dependencies, this is where they would go

In [13]:
%%file packages/sample/hello/requirements.txt
### Note: none of the following libraries is installable due to memory limits
# scikit-learn
# xgboost
# tensorflow
# onnx

### Note: the following do work
# numpy
# joblib

Writing packages/sample/hello/requirements.txt


In [14]:
!cat packages/sample/hello/requirements.txt

### Note: none of the following libraries is installable due to memory limits
# scikit-learn
# xgboost
# tensorflow
# onnx

### Note: the following do work
# numpy
# joblib

# Fit a regression model on some data

Be mindfull that digitalocean functions have low storage and memory capacity

Let's use a regression model like the one described here: https://towardsdatascience.com/ai-feynman-2-0-learning-regression-equations-from-data-3232151bd929

Specifically, the code to generate two regression functions on some sample data is found here: https://github.com/dcshapiro/AI-Feynman/blob/master/AI_Feynman_2_0.ipynb

In [15]:
from math import tan,log,exp,sqrt
from random import random

# One equation that AI Feynman discovered when trying to fit an equation to some data
def predict_version_one(x0,x1,x2,x3):
  return log(sqrt(exp(-x1 + x3))) + 3.0

# A second equation that AI Feynman discovered when trying to fit an equation to the same data
def predict_version_two(x0,x1,x2,x3):
  return -0.25*x0 - 0.25*x1 + 0.25*x2 + 0.25*x3 + 3.0

# A generator that produces data from our universe of observations. These numbers are not random. They are random *SAMPLES*
# This is the data that the regression model was designed to make predictions about
def datagen():
  while True:
    x0=random()
    x1=x0
    x2=random()
    x3=x2
    yield x0,x1,x2,x3

# Instantiate our data generator
g = datagen()

# Show that the two predict functions are predicting almost the same thing
for _ in range(5):
  x0,x1,x2,x3 = next(g)
  one = predict_version_one(x0,x1,x2,x3)
  two = predict_version_two(x0,x1,x2,x3)
  print(one-two)

0.0
0.0
0.0
-4.440892098500626e-16
0.0


Try out one of the regression models as an inference pipeline using inputs from JSON, and returning JSON

In [16]:
from math import tan,log,exp,sqrt
from random import random

# helper function to cast values from JSON strings to python floats
def argToFloat(arg,idx):
  return float(arg.get(idx, "0.0"))

# One equation that AI Feynman discovered when trying to fit an equation to some data
def predict_version_one(x0,x1,x2,x3):
  return log(sqrt(exp(-x1 + x3))) + 3.0

def main(args):
  x0,x1,x2,x3 = [argToFloat(args,"0"),
                  argToFloat(args,"1"),
                  argToFloat(args,"2"),
                  argToFloat(args,"3")]
  
  if x0!=x1 or x2!=x3 or min([x0,x1,x2,x3])<0 or max([x0,x1,x2,x3])>1:
    return {"body": "Error: Unexpected Observation"}
  
  response = predict_version_one(x0,x1,x2,x3)
  return {"body": str(response)}

######### Inference test
x0,x1,x2,x3 = next(g)

main({"0":str(x0),
      "1":str(x1),
      "2":str(x2),
      "3":str(x3)})

{'body': '3.1881090003611483'}

# Create a build script

In [17]:
%%file packages/sample/hello/build.sh
#!/bin/bash
set -e
virtualenv virtualenv
source virtualenv/bin/activate
pip install -r requirements.txt
deactivate

Writing packages/sample/hello/build.sh


In [18]:
!chmod +x packages/sample/hello/build.sh

In [19]:
!cat packages/sample/hello/build.sh

#!/bin/bash
set -e
virtualenv virtualenv
source virtualenv/bin/activate
pip install -r requirements.txt
deactivate

# Create inference pipeline

In [20]:
%%file packages/sample/hello/__main__.py
from math import tan,log,exp,sqrt
from random import random

# helper function to cast values from JSON strings to python floats
def argToFloat(arg,idx):
  return float(arg.get(idx, "0.0"))

# One equation that AI Feynman discovered when trying to fit an equation to some data
def predict_version_one(x0,x1,x2,x3):
  return log(sqrt(exp(-x1 + x3))) + 3.0

def main(args):
  x0,x1,x2,x3 = [argToFloat(args,"0"),
                  argToFloat(args,"1"),
                  argToFloat(args,"2"),
                  argToFloat(args,"3")]
  
  if x0!=x1 or x2!=x3 or min([x0,x1,x2,x3])<0 or max([x0,x1,x2,x3])>1:
    return {"body": "Error: Unexpected Observation"}
  
  response = predict_version_one(x0,x1,x2,x3)
  return {"body": str(response)}

Writing packages/sample/hello/__main__.py


In [21]:
!cat packages/sample/hello/__main__.py

from math import tan,log,exp,sqrt
from random import random

# helper function to cast values from JSON strings to python floats
def argToFloat(arg,idx):
  return float(arg.get(idx, "0.0"))

# One equation that AI Feynman discovered when trying to fit an equation to some data
def predict_version_one(x0,x1,x2,x3):
  return log(sqrt(exp(-x1 + x3))) + 3.0

def main(args):
  x0,x1,x2,x3 = [argToFloat(args,"0"),
                  argToFloat(args,"1"),
                  argToFloat(args,"2"),
                  argToFloat(args,"3")]
  
  if x0!=x1 or x2!=x3 or min([x0,x1,x2,x3])<0 or max([x0,x1,x2,x3])>1:
    return {"body": "Error: Unexpected Observation"}
  
  response = predict_version_one(x0,x1,x2,x3)
  return {"body": str(response)}

# Build and test your digitalocean function

Build

In [22]:
!doctl sandbox deploy . --remote-build

Deploying '/content/functions'
  to namespace 'fn-36f74ea7-e6c1-44dc-b004-b142654d8b0b'
  on host 'https://faas-tor1-70ca848e.doserverless.co'
Submitted action 'hello' for remote building and deployment in runtime python:default (id: ac2e1f0ef39e4865ae1f0ef39e9865c8)
Processing of 'hello' is still running remotely ...
Processing of 'hello' is still running remotely ...

Deployed functions ('doctl sbx fn get <funcName> --url' for URL):
  - sample/hello


Generate the URL for this project so that you can see it in the browser. This is also the URL we will use to post parameters to using the requests library.

In [26]:
!pip install icecream

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import requests, json
from icecream import ic
url = !doctl sbx fn get sample/hello --url
print(url[0])

x0,x1,x2,x3 = next(g)

x = { "0":str(x0),
      "1":str(x1),
      "2":str(x2),
      "3":str(x3)}
ic(x)
y = requests.post(url[0], data = x)
ic(y.text)

ic| x: {'0': '0.9430294816720703',
        '1': '0.9430294816720703',
        '2': '0.15083726450822088',
        '3': '0.15083726450822088'}


https://faas-tor1-70ca848e.doserverless.co/api/v1/web/fn-36f74ea7-e6c1-44dc-b004-b142654d8b0b/sample/hello


ic| y.text: '2.603903891418075'


'2.603903891418075'

In [30]:
y.text

'2.603903891418075'

In [58]:
with open('json_data.json', 'w') as outfile:
    json.dump(x, outfile)
!cat json_data.json

{"0": "0.9430294816720703", "1": "0.9430294816720703", "2": "0.15083726450822088", "3": "0.15083726450822088"}

We can also test directly from the command line

In [59]:
!doctl sls fn invoke sample/hello -P json_data.json

{
  "body": "2.603903891418075"
}


In [61]:
!doctl sandbox activations logs

=== af23e79994314edaa3e7999431ceda66 (success) 05/25 13:49:33 hello:0.0.10
